In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing Libraries**

In [ ]:
import numpy as np 
import pandas as pd
import os
import glob
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
#import missingno as msno
import plotly as py
import plotly.express as px
import plotly.graph_objects as go
import statistics as stat
from wordcloud import WordCloud, STOPWORDS
from plotly.subplots import make_subplots
#import folium
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

print("Libraries imported!")

Exploratory Data Analysis:- The analysis will be performed in the following order:

1.Product Data 2.District Data 3.Engagement Data.

Datasets:-

There are a total of 233 School Districts available within the data, all around USA. A school district is a geographical unit for the local administration of elementary or secondary schools in various nations.

There are a total of 372 distinct Educational Technology Products, such as tools like Meet, Zoom, Canva, educational apps like Duolingo, reading sites, or social pages like Instagram.

The data was collected between 01.01.2020 (a few months before Covid-19) until 31.12.2020. This will give a full year overview of a before the pandemic and after usage.

Engagement data was collected for all the districts with ~ 22.3M datapoints.

**Read District info**

Name Description district_id :-The unique identifier of the school district

state:- The state where the district resides in

locale :-NCES locale classification that categorizes U.S. territory into four types of areas: City, Suburban, Town, and Rural. See Locale Boundaries User's Manual for more information.

pct_black/hispanic:- Percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data

pct_free/reduced:- Percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data

countyconnectionsratio:- ratio (residential fixed high-speed connections over 200 kbps in at least one direction/households) based on the county level data from FCC From 477 (December 2018 version). See FCC data for more information.

pptotalraw:- Per-pupil total expenditure (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project. The expenditure data are school-by-school, and we use the median value to represent the expenditure of a given school district.

In [ ]:
distdata = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
distdata.head(10)

**Read Product info**

Product information data

The product file products_info.csv includes information about the characteristics of the top 372 products with most users in 2020. The categories listed in this file are part of LearnPlatform's product taxonomy.

Some products may not have labels due to being duplicate, lack of accurate url or other reasons.

Name Description LP ID:- The unique identifier of the product

URL :- Web Link to the specific product

Product Name:- Name of the specific product

Provider/Company Name:- Name of the product provider

Sector(s):- Sector of education where the product is used

Primary Essential Function:- The basic function of the product. There are two layers of labels here. Products are first labeled as one of these three categories: LC = Learning & Curriculum, CM = Classroom Management, and SDO = School & District Operations. Each of these categories have multiple sub-categories with which the products were labeled

In [ ]:
proddata = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
proddata.head(10)

**Reading Engagement data**

The data include a total of 233 school districts from across the US. For the columns pct_black/hispanic, pct_free/reduced, county_connections ratio and pp_total_raw a range is provided add Codeadd Markdown Engagement data The engagement data are aggregated at school district level, and each file in the folder engagement_data represents data from one school district*.

The 4-digit file name represents district_id which can be used to link to district information in district_info.csv.

The lp_id can be used to link to product information in product_info.csv.

Name Description time date in "YYYY-MM-DD" lp_id The unique identifier of the product pct_access Percentage of students in the district have at least one page-load event of a given product and on a given day engagement_index Total page-load events per one thousand students of a given product and on a given day

In [ ]:
path = '/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data'
files = glob.glob(os.path.join(path, "*.csv"))
files

**Combining 233 files of Engagement folder**

In [ ]:
PATH = '/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data' 
distdata = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")

temp = []

for district in distdata.district_id.unique():
    df = pd.read_csv(f'{PATH}/{district}.csv', index_col=None, header=0)
    df["district_id"] = district
    temp.append(df)
    
    
engagement = pd.concat(temp)
engagement = engagement.reset_index(drop=True)

In [ ]:
engagement.shape

**The number of unique providers from product.info file**

In [ ]:


print('Total unique products : ', proddata['LP ID'].unique().size)
print('Total unique providers in top 372 products are : ',proddata['Provider/Company Name'].unique().size)
print('Total unique sectors are : ',proddata['Sector(s)'].unique().size)
print('They are : ')
a=list(proddata['Sector(s)'].unique())
for i in a:
    print(i)

In [ ]:
# python libraries to study missing data
from wordcloud import WordCloud, STOPWORDS
import missingno as msno

In [ ]:
def missing_values(df):
    ''' This function takes a data frame as input 
    prints the fraction of entries with missing values (NaN)
    prints the list of columns with corresponding number of missing values
    '''
    # Total number of entries (rows X columns) in the dataset
    total= df.size
    #Number of missing values per column
    missingCount = df.isnull().sum()
    #Total number of missing values
    missing_tot = missingCount.sum()
    # Calculate percentage of missing values
    print('Total number of missing values for each column of dataframe: \n \b \b \b',missingCount)
    print("The dataset contains", round(((missing_tot/total) * 100), 2), "%", "missing values")
    print('Total number of rows with at least one missing value column are ',df[df.isnull().any(axis=1)].shape[0])
    print('% of rows with missing data ',round(((df[df.isnull().any(axis=1)].shape[0]/df.shape[0])*100),2),'%\n\n')
    
    
def column_missingdata(df):
    ''' This function takes a data frame as input 
    prints the list of columns with corresponding % of missing values
    '''
    #check for missing values per column
    values=df.isnull().sum()#.sort_values(ascending=False)
    total= df.size
    #percentage of missing values per column
    percentage=(values/total) * 100
    print('% of missing values for each column of dataframe: \n \b \b \b',percentage,'\n\n\n')

def plot_missingdata(df:pd.DataFrame, title:str, xlabel:str, ylabel:str):
    ''' This function takes a data frame as input 
    plots the list of columns with corresponding total number of missing values
    '''
    # Let us see what columns have missing values
    # total number of missing values for each dataframe column
    missing = df.isnull().sum() 
    # keeping only the columns with missing values>0 
    missing = missing[missing > 0] 
    # sorting in order of missing values and making the change to original missing series
    missing.sort_values(inplace=True) 
    missing.plot.bar()
    plt.title(title, size=15,loc='left')
    plt.xticks(fontsize=11,rotation=45)
    plt.yticks(fontsize=11)
    plt.xlabel(xlabel, fontsize=13)
    plt.ylabel(ylabel, fontsize=13)
    plt.show()
    
    
def fix_missing_mean(df,col):
    ''' This function takes a data frame and column name as input 
    replaces the missing values of a particular column with it's mean value
    '''
    #replace missing values with mean 
    df[col].fillna(df[col].mean(), inplace = True)    

def fix_missing_mode(df,col):
    ''' This function takes a data frame and column name as input 
    replaces the missing values of a particular column with it's mode value
    '''
    #replace missing values with mean 
    df[col].fillna(df[col].mode(), inplace = True)    

    
def fix_missing_ffill(df, col):
    ''' This function takes a data frame and column name as input 
    replaces the missing values of a particular column with the value from the previous row
    '''
    df[col] = df[col].fillna(method='ffill')  
    
def fix_missing_bfill(df, col):
    ''' This function takes a data frame as input 
    replaces the missing values of a particular column with the value from the next row
    '''
    df[col] = df[col].fillna(method='bfill')

**Missing Values of district.info file**

In [ ]:
#missing values in district
missing_values(distdata)
column_missingdata(distdata)
plot_missingdata(distdata,'districts dataset columns with missing values','Column Name','No. of Missing values')

In [ ]:
distdata.shape

The heatmap below shows correlation between the missing tuples

In [ ]:
#heat map of missing value , it will show the corrrelation between different columns for missing values
ax = msno.heatmap(distdata,figsize=(6,6))
plt.show()

From above heatmap, we can conclude that 'state','locale','pct_black/hispanic' are correlated with each other. Hence we can drop them.


In [ ]:
#district
#drop nan values in state column
distdata_x = distdata[distdata["locale"].notna()].reset_index(drop=True)
#check missing columns in district dataset
column_missingdata(distdata_x)

In [ ]:
distdata_x.shape

In [ ]:
distdata_x.head(10)

In [ ]:
missing_values(distdata_x)

In [ ]:
distdata_x.shape

Using forward fill method to fill in the empty values.

In [ ]:
fix_missing_ffill(distdata_x,'pct_free/reduced')
fix_missing_ffill(distdata_x,'county_connections_ratio')
fix_missing_ffill(distdata_x,'pp_total_raw') 

In [ ]:
distdata_x.shape

In [ ]:
distdata_x.head()

In [ ]:
distdata_x.shape

In [ ]:
missing_values(distdata_x)

**Missing values in products.info file**

In [ ]:
#missing values in products
missing_values(proddata)
column_missingdata(proddata)
plot_missingdata(proddata,'products dataset columns with missing values','Column Name','No. of Missing values')

The heatmap below shows correlation between the missing tuples

In [ ]:
#heat map of missing value , it will show the corrrelation between different columns for missing values
ax = msno.heatmap(proddata,figsize=(6,6))
plt.show()

From above heat map, we can conclude that
1. Sector(s) and Provider/Company name, Primary Essential Function and Provider/Company name are closely correlated.
2.  Primary Essential Function and Sector(s) are higly correlated.

In [ ]:
# dropping all the rows with any missing values 
proddata_x=proddata
proddata_x.dropna(axis=0,how='any',inplace=True)

#check missing columns in product dataset
column_missingdata(proddata_x)

In [ ]:
proddata_x.tail(5)

In [ ]:
proddata_x.shape

**Missing values in engagement**

In [ ]:
#missing values in engagement
missing_values(engagement)
column_missingdata(engagement)
plot_missingdata(engagement,'engagement dataset columns with missing values\n\n','Column Name','No. of Missing values')

The heatmap below shows correlation between the missing tuples

In [ ]:
#heat map of missing value , it will show the corrrelation between different columns for missing values
ax = msno.heatmap(engagement,figsize=(6,6))
plt.show()

From the above heat map, we can conclude that 
1. pct_access and lp_id, engagement_index and lp_id are very lightly correlated as the colour of heat map is slightly pinkish, but nearly equal to white(0.00).
2. engagement_index and pct_access are also lightly correlated as the colour of the heat map is slightly bluish, but nearly equal to white(0.00).


In [ ]:
#drop nan values in lp_id column
engagement_x = engagement[engagement["lp_id"].notna()].reset_index(drop=True)

# fill the missing values with mean value of the column in the engagement dataset
fix_missing_mean(engagement_x,'pct_access')
fix_missing_mean(engagement_x,'engagement_index')

In [ ]:
engagement.head()

In [ ]:
engagement_x.head()

In [ ]:
engagement_x.shape

In [ ]:
engagement.shape

In [ ]:
#check missing columns in enagement dataset
column_missingdata(engagement_x)

**ANALYSIS**

In [ ]:
#defining functions for plots
#countplot
def plot_count(df:pd.DataFrame, y_col:str, title:str, xlabel:str, ylabel:str):
    fig, ax = plt.subplots(1, 1, figsize=(10,6), sharey=True)
    sns.countplot(data=df, y=y_col,edgecolor="white",palette="viridis",order=df[y_col].value_counts().index)
    total = df[y_col].value_counts().sum()
    for i, v in enumerate(df[y_col].value_counts().sort_values(ascending=False).values): 
        frac1 = (v/total)*100
        frac = "{:.2f}".format(frac1)
        v1 = str(v)+' ('+str(frac)+'%)'
        ax.text(v*1.01, i, v1,fontsize=10,color='black',weight='bold')
   
    plt.title(title, size=20)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.show()
    
#pieplot_for locale
def plot_pie(df:pd.DataFrame,column:str, title:str):
    fig, ax  = plt.subplots(figsize=(16, 8))
    fig.suptitle(column+' Distribution', font="Serif",fontsize=20)
    explode = (0.01, 0.01, 0.01, 0.01) 
    labels = list(df[column].value_counts().index)
    sizes = df[column].value_counts().values
    ax.pie(sizes, 
           explode=explode,
           startangle=60, 
           labels=labels,
           autopct='%1.0f%%', 
           pctdistance=0.7)
    ax.add_artist(plt.Circle((0,0),0.5,fc='white'))
plt.show()
    
#barplot   
def plot_bar(df:pd.DataFrame,minimum:str,maximum:str,y_col:str, title:str, xlabel:str, ylabel:str):
    df1 = df.groupby(y_col)[minimum, maximum].mean().sort_values([minimum, maximum],ascending=False)
    df1.plot(kind = 'bar',color=['blue','darkblue'])
    plt.title(title, size=20)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.legend(bbox_to_anchor=(1.05, 1), borderaxespad=0)
    plt.show()


def plot_barh(df:pd.DataFrame, x_col:pd.DataFrame,y_col:pd.DataFrame, title:str, xlabel:str,ylabel:str):
    fig, ax  = plt.subplots(figsize=(10,6))    
    sns.barplot(data = df,y=y_col,x=x_col,palette="viridis")
    plt.title(title, size=20)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.show()
    
def plot_bar1(df:pd.DataFrame, x_col:pd.DataFrame,y_col:pd.DataFrame, title:str, xlabel:str,ylabel:str):
    fig, ax  = plt.subplots(figsize=(10,6)) 
    sns.barplot(data = df,y=y_col,x=x_col,palette="viridis",order=df.groupby(y_col).mean()[x_col].sort_values(ascending=False).index,ci=None)
    plt.title(title, size=20)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.show()
    
    
def plot_bar2(df1:pd.DataFrame, x_col:str,y_col1:str, ax1_title:str, xlabel1:str,ylabel1:str,df2:pd.DataFrame,y_col2:str,ax2_title:str, xlabel2:str,ylabel2:str):
    fig, ax = plt.subplots(1, 2, figsize=(12,5))
    sns.barplot(data = df1,y=y_col1,x=x_col,palette="viridis", ax=ax[0])
    ax[0].set_title(ax1_title)
    ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
    ax[0].set_xlabel(xlabel1)
    ax[0].set_ylabel(ylabel1)

    sns.barplot(data = df2,y=y_col2,x=x_col,palette="viridis", ax=ax[1])
    ax[1].set_title(ax2_title)
    ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
    ax[1].set_xlabel(xlabel2)
    ax[1].set_ylabel(ylabel2)
    plt.show()    
    
def plot_scatter(df: pd.DataFrame, x_col: str, y_col: str, title: str, hue: str):
    plt.figure(figsize=(10, 5))
    sns.scatterplot(data = df, x=x_col, y=y_col, hue=hue, size=hue,sizes=(20, 200))
    plt.title(title, size=20)
    plt.xticks(fontsize=14)
    plt.yticks( fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), borderaxespad=0)
    plt.show() 
    
def plot_pairplot(df: pd.DataFrame):#, hue:str, diag_kind:str
    sns.pairplot(df)
    #pd.plotting.scatter_matrix(df, alpha=0.2)
    plt.show() 
    
    
def plot_line(df: pd.DataFrame,x_col: str, y_col: str, hue: str, title:str, xlabel:str, ylabel:str, text1:int, text2:int):#, hue:str, diag_kind:str
    fig, ax  = plt.subplots(figsize=(15,7))    
    sns.lineplot(data=df, x=x_col, y=y_col, hue=hue,lw=2)
    plt.title(title, size=30)
    plt.xticks(fontsize=20)#,rotation=45)
    plt.yticks(fontsize=20)
    plt.xlabel(xlabel, fontsize=30)
    plt.ylabel(ylabel, fontsize=30)
    plt.legend(bbox_to_anchor=(1.05, 1), borderaxespad=0)
    plt.axvline(pd.Timestamp('2020-03-11'),color='black')
    ax.text(pd.Timestamp("2020-03-11"), text1, "  WHO has declared Covid-19 a pandemic",size=20,fontname = 'monospace',color='black')
    xy = datetime(2020,6,1,0,0,0), ax.get_ylim()[0]
    w, h = timedelta(days=92), ax.get_ylim()[1] - ax.get_ylim()[0]
    ax.add_patch(patches.Rectangle(xy, w, h,color ='black',alpha=0.1))
    ax.text(pd.Timestamp("2020-06-17"), text2, "Summer Break",size=20,fontname = 'monospace',color='black')
    plt.show() 
    
def plot_line_day(df: pd.DataFrame,x_col: str, y_col: str, hue: str, title:str, xlabel:str, ylabel:str, text1:int, text2:int):#, hue:str, diag_kind:str
    fig, ax  = plt.subplots(figsize=(15,7))    
    sns.lineplot(data=df, x=x_col, y=y_col, hue=hue,style=hue,ci=None,markers=False, dashes=False, lw=2)
    plt.title(title, size=30)
    plt.xticks(fontsize=20)#,rotation=45)
    plt.yticks(fontsize=20)
    plt.xlabel(xlabel, fontsize=30)
    plt.ylabel(ylabel, fontsize=30)
    plt.legend(bbox_to_anchor=(1.05, 1), borderaxespad=0)
    plt.show()

**Plot for Number of districts per state** 

In [ ]:
plot_count(distdata_x,'state','Number of districts per state','Districts Count','State')

**Plot for Number of districts per locale**

In [ ]:
#barplot
plot_count(distdata_x,'locale','Number of districts per locale type','Districts count for locale type','Locale')

In [ ]:
#pie plot
plot_pie(distdata_x,'locale','Number of districts per locale')

The intervals of pct_black/hispanic, pct_free/reduced, county_connections_ratio and pp_total_raw are split into two columns based on their minimum and maximum values.

In [ ]:
distdata_y=distdata_x
#'pct_black/hispanic'
distdata_y[['min_pct_black/hispanic','max_pct_black/hispanic']] = distdata_y['pct_black/hispanic'].str.split(",",expand=True)
distdata_y['min_pct_black/hispanic'] = distdata_y['min_pct_black/hispanic'].str.strip('[')
distdata_y['max_pct_black/hispanic'] = distdata_y['max_pct_black/hispanic'].str.strip('[')

#'pct_free/reduced'
distdata_y[['min_pct_free/reduced','max_pct_free/reduced']] = distdata_y['pct_free/reduced'].str.split(",",expand=True)
distdata_y['min_pct_free/reduced'] = distdata_y['min_pct_free/reduced'].str.strip('[')
distdata_y['max_pct_free/reduced'] = distdata_y['max_pct_free/reduced'].str.strip('[')

#'county_connections_ratio'
distdata_y[['min_county_connections_ratio','max_county_connections_ratio']] = distdata_y['county_connections_ratio'].str.split(",",expand=True)
distdata_y['min_county_connections_ratio'] = distdata_y['min_county_connections_ratio'].str.strip('[')
distdata_y['max_county_connections_ratio'] = distdata_y['max_county_connections_ratio'].str.strip('[')

#'pp_total_raw'
distdata_y[['min_pp_total_raw','max_pp_total_raw']] = distdata_y['pp_total_raw'].str.split(",",expand=True)
distdata_y['min_pp_total_raw'] = distdata_y['min_pp_total_raw'].str.strip('[')
distdata_y['max_pp_total_raw'] = distdata_y['max_pp_total_raw'].str.strip('[')

#import the original columns
distdata_y.drop(['pct_black/hispanic', 'pct_free/reduced', 'county_connections_ratio', 'pp_total_raw'], axis = 1, inplace = True)

In [ ]:
distdata_y.dtypes

In [ ]:
#convert object to numbers
cols = distdata_y.columns.drop(['state', 'locale'])
distdata_y[cols] = distdata_y[cols].apply(pd.to_numeric, errors = 'coerce')

In [ ]:
#except state and locale we have converted all others object columns to numeric columns
distdata_y.dtypes

In [ ]:
plot_bar(distdata_y,'min_pct_black/hispanic','max_pct_black/hispanic','locale','Mean % of black or hispanic students by locale','Locale','Mean')
plot_bar(distdata_y,'min_pct_free/reduced','max_pct_free/reduced','locale','Mean % of students eligible for free/reduced lunch by locale','Locale','Mean')
plot_bar(distdata_y,'min_pp_total_raw','max_pp_total_raw','locale','Mean total expenditure in USD per student by locale','Locale','Mean')
plot_bar(distdata_y,'min_county_connections_ratio','max_county_connections_ratio','locale','Mean connection ratio by locale','Locale','Mean')

In [ ]:
distdata_y["avg_pct_black/hispanic"] = distdata_y[["min_pct_black/hispanic", "max_pct_black/hispanic"]].mean(axis=1)
distdata_y["avg_pct_free/reduced"] = distdata_y[["min_pct_free/reduced", "max_pct_free/reduced"]].mean(axis=1)
distdata_y["avg_pp_total_raw"] = distdata_y[["min_pp_total_raw", "max_pp_total_raw"]].mean(axis=1)
distdata_y["avg_county_connections_ratio"] = distdata_y[["min_county_connections_ratio", "max_county_connections_ratio"]].mean(axis=1)

**Mean percentage of black/hispanic students per state**

In [ ]:
plot_bar1(distdata_y,'avg_pct_black/hispanic','state','Mean % of black or hispanic students per State','Mean Student Expenditure','State')


**Mean percentage of free/reduced lunch per state**

In [ ]:
plot_bar1(distdata_y,'avg_pct_free/reduced','state','Mean % of students eligible for free/reduced lunch per State','Mean Student Expenditure','State')


**Mean per pupil expenditure per state**

In [ ]:
plot_bar1(distdata_y,'avg_pp_total_raw','state','Mean Student Expenditure per State','Mean Student Expenditure','State')


**Mean connection ratio per state**

In [ ]:
plot_bar1(distdata_y,'avg_county_connections_ratio','state','Mean Connection Ratio per State','Mean Connection Ratio','State')

**Distribution of different sectors : PreK-12, Higher and Corporate.**

In [ ]:
s1=0
s2=0
s3=0
for s in proddata["Sector(s)"]:
    if(not pd.isnull(s)):
        s = s.split(";")
        for i in range(len(s)):
            sub = s[i].strip()
            if(sub == 'PreK-12'): 
                s1=s1+1
            if(sub == 'Higher Ed'): 
                s2=s2+1
            if(sub == 'Corporate'): 
                s3=s3+1

fig, ax  = plt.subplots(figsize=(14, 8))
fig.suptitle('Sector Distribution', font="Serif",fontsize=20)
explode = (0.01, 0.01, 0.01)#,0.01,0.01)

labels = ['PreK-12','Higher Ed','Corporate']
sizes = [s1,s2,s3]
ax.pie(sizes,
       startangle=60,
       explode=explode, 
       labels=['PreK-12','Higher Ed','Corporate'],
       autopct='%1.2f%%', 
       pctdistance=0.7)
ax.add_artist(plt.Circle((0,0),0.5,fc='white'))
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,20), sharey=True)
sns.countplot(y = 'Primary Essential Function', 
              data = proddata_x, 
              edgecolor="white",
              palette="viridis",
              order = proddata['Primary Essential Function'].value_counts().index)
 

total = proddata_x["Primary Essential Function"].value_counts().sum()
for i, v in enumerate(proddata_x["Primary Essential Function"].value_counts().sort_values(ascending=False).values): 
    frac1 = (v/total)*100
    frac = "{:.2f}".format(frac1)
    v1 = str(v)+' ('+str(frac)+'%)'
    ax.text(v*1.01, i, v1,fontsize=10,color='black',weight='bold')
    
plt.xticks(size=15)
plt.yticks(size=15)
plt.ylabel("Primary Essential Function",size=20)
plt.xlabel("products count",size=20)
plt.title('Number of products used by Primary Essential Function',size=25)

plt.show()

We can infer from above, that Learning Curriculum - Digital Learning Platforms use most of the products to facilitate digital learning.

**Distribution of Primary Essential Function**

In [ ]:
s1=0
s2=0
s3=0
for s in proddata_x["Primary Essential Function"]:
    if(not pd.isnull(s)):
        s1 = s.count("CM")+1
        s2 = s.count("LC")+1
        s3 = s.count("SDO")+1

fig, ax  = plt.subplots(figsize=(16, 10))
fig.suptitle('% Distribution of primary essential function', font="Serif",fontsize=20)
explode = (0.01, 0.01, 0.01)#,0.01,0.01)
labels = ['CM','LC','SDO']
sizes = [s1,s2,s3]
ax.pie(sizes,
       startangle=60,
       explode=explode, 
       labels=labels,
       autopct='%1.2f%%', 
       pctdistance=0.7)
ax.add_artist(plt.Circle((0,0),0.5,fc='white'))
subgroup_names_legs=['CM:Classroom Management', 'LC:Learning & Curriculum', 'SDO:School & District Operations']
plt.legend(subgroup_names_legs,loc='best')
plt.show()

In [ ]:
#Subdivide primary essential function
proddata_y=proddata_x
proddata_y['primary_function_main'] = proddata_y['Primary Essential Function'].apply(lambda x: x.split(' - ')[0] if x == x else x)
proddata_y['primary_function_sub'] = proddata_y['Primary Essential Function'].apply(lambda x: x.split(' - ')[1] if x == x else x)

# Synchronize similar values
proddata_y['primary_function_sub'] = proddata_y['primary_function_sub'].replace({'Sites, Resources & References' : 'Sites, Resources & Reference'})
# Deleting the 'Primary Essential Function' column from the product dataset as it is no longer needed since we already have split it into two sub-columns.
proddata_y.drop("Primary Essential Function", axis=1, inplace=True)

**C1 : What is the picture of Digital Connectivity and Engagement in 2020?**

**Plot to show student Engagement during the course of pandemic**

In [ ]:
engagement['time'] = pd.to_datetime(engagement['time'])
engagement['month'] = engagement['time'].dt.month

fig, ax = plt.subplots(1, 1, figsize=(7,5), sharey=True)
sns.barplot(x = 'month',y='engagement_index', 
              data = engagement.groupby(engagement['month']).mean().reset_index(), 
              edgecolor="white",
              palette="viridis")


plt.title('Mean engagement index vs month',size=15)
plt.xlabel("Month",size=15)
plt.ylabel("Mean engagement index",size=15)
plt.show()

Above graph tells us that the engagement during the months of June-July decreased marking the summer breaks. Further rise in the months of August-September mark the beginning of new academic year.

In [ ]:
#combining the three tables
engagement_x['lp_id'] = engagement_x['lp_id'].astype('int')
engagement_x['district_id'] = engagement_x['district_id'].astype('int')

# merging engagement and district data based on district id 
Data = pd.merge(engagement_x,distdata_x,how='left',on='district_id')

# merging above data with prod data based on LP ID (unique product identifier)
Data = pd.merge(Data,proddata_y,how='left',left_on='lp_id',right_on='LP ID')

In [ ]:
#dropping missing values from the dataset
#Data.dropna(axis=0,how='any',inplace=True)
Data = Data[Data["LP ID"].notna()].reset_index(drop=True)

In [ ]:
Data.head()

**Plot to identify the number of unique days provided in the data set**

In [ ]:
plt.figure(figsize=(7,4))
sns.histplot(Data.groupby('district_id').time.nunique(), bins=40, color = 'blue')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Number of days', fontsize=10)
plt.ylabel('Number of districts', fontsize=10)
plt.title('Districts with unique days of engagement ', fontsize = 15)

plt.show()

Most of the districts have the data of complete year of 2020(366 days).

**C2 : What is the effect of the COVID-19 pandemic on online and distance learning, and how might this also evolve in the future?**

**Plot for mean engagement index and percentage of access per state**

In [ ]:
Datax=Data.loc[((Data["Sector(s)"]=="PreK-12")|(Data["Sector(s)"]=="PreK-12; Higher Ed")|(Data["Sector(s)"]=="PreK-12; Higher Ed; Corporate"))&(Data["primary_function_main"]=="LC")]
state_eng=pd.DataFrame(Datax.groupby('state')[['engagement_index','avg_pp_total_raw','avg_pct_black/hispanic','avg_pct_free/reduced']].mean().sort_values(['engagement_index','avg_pp_total_raw','avg_pct_black/hispanic','avg_pct_free/reduced'],ascending=False))
state_eng=state_eng.reset_index()
plot_barh(state_eng,'engagement_index','state','Mean Engagement Index per State','Engagement','State')

state_access=pd.DataFrame(Datax.groupby('state')[['pct_access','avg_pp_total_raw','avg_pct_black/hispanic','avg_pct_free/reduced']].mean().sort_values(['pct_access','avg_pp_total_raw','avg_pct_black/hispanic','avg_pct_free/reduced'],ascending=False))
state_access=state_access.reset_index()
plot_barh(state_access,'pct_access','state','Mean pct_access per State','Access','State')

Inference of above graph:
Change of student engagement with different types of education technology over the course of the pandemic:

Engagement index for SDO : School and district operations is comparatively higher.

**C3 : How does student engagement with different types of education technology change over the course of pandemic?**

**Top 5 companies which provided the most of the products to facilitate digital learning**

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,5), sharey=True)
sns.countplot(y = 'Provider/Company Name', 
              data = proddata_x, 
              edgecolor="white",
              palette="viridis",#"rocket_r",
              order = proddata['Provider/Company Name'].value_counts().index[:5])

for rect in ax.patches:
    ax.text (rect.get_width(), rect.get_y() + rect.get_height() / 2,"%.1f"% rect.get_width(), weight='bold' )
 
 ###
plt.xticks(size=10)
plt.yticks(size=10)
plt.ylabel("Provider/Company Name",size=15)
plt.xlabel("products counts",size=15)
plt.title('Number of products per Provider/Company Names (Top 5)',size=15)

plt.show()


Above graph tells that due to shifting of learning to digital mode, Google LLC, Houghtom Mifflin Harcourt, Microsoft are the top 3 companies widely used during the course of pandemic. 

**Plots for showing the student engagement in different Primary Essential Functions**

In [ ]:
# for our plot we only need following columns: time, engagement_index, primary_function_main 
Data['time'] = pd.to_datetime(Data['time'])
Data['month'] = Data['time'].dt.month

sns.catplot(x = 'month',y='engagement_index', 
              data = Data.groupby(by=['month','primary_function_main']).mean().reset_index(), 
              edgecolor="white",kind='bar',
              palette="viridis",col='primary_function_main',col_wrap=2,height=4)


plt.show()

**Corelation Graphs for different attributes**

In [ ]:
# Correlation between different attributes.
# Plotting correlation graphs using Pearson's correlation coefficient.
#g = sns.pairplot(state_access)

from scipy.stats import pearsonr
def corrfunc(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    ax.annotate(f'ρ = {r:.2f}', xy=(.6, .9), xycoords=ax.transAxes)

#g.map_offdiag(corrfunc)
#plt.show()
# lets study the engagement correlation with 
g=sns.pairplot(state_eng)
g.map_offdiag(corrfunc)
plt.show()

Conclusion for the above plot:

1. We observe a positive correlation between engagement index and per pupil total expenditure. This implies that if expenditure increases, we can expect a significant increase in engagement of students.

2. We find a negative correlaton between engagement index and average percentage of people identified as black or hispanic because, if the number of people identified as black or hispanic is more, then the engagement is less as they are economically weaker compared to others.

3. There is a strong negative correlation observed between engagement index and average percentage of people getting free or reduced lunch as people get concessions on the prices of lunch based upon their economic status. Hence, if engagement index is more then it implies that the students are not economically weak.

4. We can also conclude that black/hispanic communities also have lower engagement index

In [ ]:
sns.relplot(x="avg_pp_total_raw", y="engagement_index", kind="line", data=state_eng,ci=None)

The above graph tells us that if the average per pupil expenditure is less, then the expenditure is also less.
But as it increases, there is significant increase in engagement index

In [ ]:
sns.relplot(x="avg_pct_black/hispanic", y="engagement_index", kind="line", data=state_eng,ci=None)


The above plot says that if the average percentage of people identified as black or hispanic is less than 50%, then the engagement is less. But if the people identified as black or hispanic are in the majority, then consequently the engagement index of students increases.

In [ ]:
sns.relplot(x="avg_pct_free/reduced", y="engagement_index", kind="line", data=state_eng,ci=None)


In the above graph, we can observe that if percentage of students getting free or reduced lunch prices is less, then the engagement is high as they are not economically weak. But as the percentage of students getting free or reduced lunch prices increases, then their engegement decreases.

In [ ]:
sns.relplot(x="avg_pct_black/hispanic", y="avg_pct_free/reduced", kind="line", data=state_eng,ci=None)

From above graph we can conclude that, if the average percentage of people identified as black or hispanic students is less than 50%, then most of them will be getting free or reduced lunch prices as they will be the minority.
Similarly, if they are more than 50%, then they will be considered as majority and their percentage of getting free or reduced lunch prices will go down.


**Q5. Do certain state interventions, practices or policies (e.g., stimulus, reopening, eviction moratorium) correlate with the increase or decrease online engagement?**

1. From the analysis, we can conclude that engagement index of Black/Hispanic communities is comparatively less as they belong to economically weaker community.
2. State interventions like raising funds for economically weaker communities can result in a significant growth in the engagement index of students.
3. Evolution of different technologies and products can lead to a tremendous increase in the engagement of students not only in US , but also worldwide.


**Extra analysis**

In [ ]:
# lets check the correlation between different features of data or variables , 
# pct access, expenditure, black/hispanic, free/reduced lunch eligibility %
g = sns.pairplot(state_access)

# let's show pearson correlation coefficient (https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) 
#above each pair plot 
from scipy.stats import pearsonr
def corrfunc(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    ax.annotate(f'ρ = {r:.2f}', xy=(.6, .9), xycoords=ax.transAxes)

g.map_offdiag(corrfunc)
plt.show()